In [8]:
%load_ext autoreload
%autoreload 2
%load_ext memory_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import os
from tempfile import mkdtemp
import numpy as np
import matplotlib.pyplot as plt

from ics.hxutils import hxramp

from h4rg_analysis import ramputils,io

plt.rcParams['figure.figsize'] = (8,8)
bestbaselines = open ('../data/paths/best_ever_baseline_darks_8_21_2022.txt', 'r').read().splitlines()[1:]

In [7]:
pt = io.jhu2tiger ( bestbaselines[1] )
tag = os.path.basename(pt).strip('.fits')

In [39]:
%%memit -r1
ramp = hxramp.HxRamp( pt )

peak memory: 9707.54 MiB, increment: 0.01 MiB


In [16]:
tempdir = mkdtemp ()
tempname = '/'.join([tempdir, 'ramptemp.dat'])

In [40]:
fp = np.memmap ( tempname, dtype='u2', mode='w+', shape=(ramp.nreads, ramp.nrows, ramp.ncols ))

In [20]:
r0=0
r1=-1

In [21]:
r0 = ramp._readIdxToAbsoluteIdx ( r0 )
r1 = ramp._readIdxToAbsoluteIdx ( r1 )

In [63]:
import psutil

In [64]:
%%memit -r1
for r_i in np.arange(r0, r1+1):
    n = ramp._readIdxToFITSIdx ( r_i )
    extname = f'REF_{n}'
    irpImage = ramp.fits[extname].read ()
    irpImage = hxramp.constructFullIrp(irpImage, ramp.nchan, refPix=ramp.interleaveOffset)
    fp[r_i] = irpImage
    #fp.flush ()
    break


peak memory: 267.64 MiB, increment: 63.81 MiB


In [72]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss/1e9)  # in bytes 

0.21372928
